In [2]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os

In [5]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [6]:
doc=read_doc('documents/')
len(doc)

0

In [7]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [8]:
documents=chunk_data(docs=doc)
len(documents)

0

In [10]:
%pip install sentence-transformers

from sentence_transformers import SentenceTransformer

# Load a pre-trained model from Hugging Face
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to embed a query
def embed_query(query):
    return model.encode([query])[0]

# Example usage
query = "How are you?"
vectors = embed_query(query)
print(vectors)
print("Length of vectors:", len(vectors))



Note: you may need to restart the kernel to use updated packages.Collecting sentence-transformers
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     ----------------------------------- -- 41.0/43.6 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 428.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached Mar

c:\Users\SHILPA\Desktop\git\end to end vectr db\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SHILPA\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[ 7.00386195e-03  1.09141301e-02  8.74624997e-02  8.67992863e-02
  2.66484693e-02 -6.75052255e-02  7.26844966e-02 -2.51548439e-02
 -8.26884285e-02  1.61130670e-02 -5.94526529e-03  2.16282923e-02
 -1.47851398e-02  2.62616593e-02  4.37727682e-02 -5.28110340e-02
  5.88161461e-02 -1.28315508e-01 -9.48690474e-02  3.96221504e-02
 -1.13020159e-01  3.81748118e-02  2.94629745e-02  1.15779653e-01
 -2.65442841e-02 -4.39773453e-03 -4.28429097e-02 -7.10976683e-03
  2.19259728e-02 -6.68361038e-02 -1.27751499e-01  4.21850793e-02
 -8.64470154e-02 -3.36370990e-02 -3.87000218e-02 -1.20038539e-02
 -1.38557339e-02 -1.09099329e-01 -1.55588472e-02  9.30230785e-03
  3.18632051e-02 -3.02896108e-02 -1.87066142e-02 -3.09348181e-02
  7.75421560e-02 -3.62924971e-02  1.68967973e-02  4.19577919e-02
  9.20711905e-02  2.53914911e-02 -9.65331867e-02 -3.95974852e-02
 -4.53626849e-02  2.29310971e-02 -4.80529480e-03 -7.83914421e-03
 -4.89539355e-02  5.84354065e-02  5.89098185e-02 -3.11468765e-02
  5.26644513e-02  5.85511

In [20]:
%pip install sentence-transformers PyPDF2


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [21]:
import PyPDF2
from sentence_transformers import SentenceTransformer

# Load a pre-trained model from Hugging Face
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to embed a query
def embed_query(query):
    return model.encode([query])[0]

# Function to read and extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Example PDF file path (replace with your actual PDF file path)
pdf_path = "budget_speech.pdf"

# Extract text from the PDF
document_text = extract_text_from_pdf(pdf_path)

# Generate embeddings for the extracted text
embeddings = model.encode([document_text])[0]

# Print the embeddings
print(embeddings)
print("Length of embeddings:", len(embeddings))


[-5.54053970e-02  7.33296061e-03 -9.22871195e-03 -1.22442124e-02
 -6.72087725e-03  3.53260860e-02  3.07476763e-02  3.40156294e-02
 -8.22332725e-02  3.82587872e-02  2.18511024e-03 -6.45571873e-02
 -4.63764966e-02 -3.49178836e-02  5.38254762e-03 -3.04681505e-03
  4.55934294e-02 -6.37075212e-03 -6.08499199e-02 -1.03888046e-02
  1.20601721e-01  6.16486110e-02  2.81938612e-02 -5.65461218e-02
 -1.30413286e-02  2.42085736e-02 -2.15761922e-02 -1.00049920e-01
 -2.64662638e-04 -9.63587966e-03  4.61504981e-02  7.78295845e-02
 -5.39878793e-02 -5.22057433e-03  7.06641227e-02 -1.11278258e-02
 -2.58823112e-02  9.09516364e-02  6.42531812e-02 -6.99622631e-02
 -1.95941180e-02 -4.53546196e-02 -6.57360256e-02 -2.70720813e-02
 -1.89060140e-02 -8.53327885e-02  6.07419014e-02 -5.97756058e-02
 -4.75268327e-02  1.74803901e-02 -2.13624481e-02 -8.39773472e-03
  1.43309375e-02 -1.90572124e-02  2.72024833e-02 -1.28069058e-01
 -3.80055867e-02 -2.90544536e-02  4.20551859e-02 -1.74516160e-02
 -4.97963242e-02 -1.87544

In [11]:
## Embedding Technique Of OPENAI
# embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
# embeddings

In [13]:
# vectors=embeddings.embed_query("How are you?")
# len(vectors)

1536

In [28]:
%pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [38]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="your_pinecone_api")

In [39]:
pc.create_index(
    name="your_pinecone_name",
    dimension=2, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '88f66b11d9c4153f5f99befa8b347bed', 'Date': 'Sat, 13 Jul 2024 16:12:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [17]:
# ## Vector Search DB In Pinecone
# pinecone.init(
#     api_key="14642a22-4af9-4d20-8d09-5c974e7093a1",
#     environment="Serverless"
# )
# index_name="vectorlangchain"

In [40]:
index=Pinecone.from_documents(doc,embeddings,index_name="your_pinecone_name")

AttributeError: from_documents is not a top-level attribute of the Pinecone class provided by pinecone's official python package developed at https://github.com/pinecone-io/pinecone-python-client. You may have a name collision with an export from another dependency in your project that wraps Pinecone functionality and exports a similarly named class. Please refer to the following knowledge base article for more information: https://docs.pinecone.io/troubleshooting/pinecone-attribute-errors-with-langchain


In [32]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [33]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [34]:
llm = SentenceTransformer('all-MiniLM-L6-v2')  # Replace with your desired model
chain=load_qa_chain(llm,chain_type="stuff")

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [19]:
# llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
# chain=load_qa_chain(llm,chain_type="stuff")

In [35]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [41]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

NameError: name 'index' is not defined